In [1]:
import os, pickle, glob
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
#from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType
import pandas as pd

In [2]:
sc = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
	.config("spark.executor.memory", "10g") \
    .config('spark.executor.instances', 5) \
	.getOrCreate()

In [ ]:
#!unzip ./traffic_congestion.zip
csv_file = "./us_congestion_2016_2022/us_congestion_2016_2022.csv"
df = sc.read.csv(csv_file, header=True, inferSchema=True)

In [ ]:
first_row = df.take(1)[0]
print(first_row)

In [ ]:
#Filter out specific fields we want to keep
fields_to_keep = ['ID', 'Severity', 'DelayFromFreeFlowSpeed(mins)', 'StartTime', 'Congestion_Speed', 'State', 'Pressure(in)', 'Visibility(mi)', 'WindSpeed(mph)', 'Precipitation(in)', 'Weather_Event']

# 33m to 5m data if I filter out the 5 min or longer for traffic
filtered_df = df.select(*fields_to_keep).where(col("State").isNotNull()).where(col("DelayFromFreeFlowSpeed(mins)") > 5.0)
filtered_df.show()

In [ ]:
state_count_df = filtered_df.groupBy("State").count()
state_count_df.show()

In [ ]:
pdf = filtered_df.toPandas()